<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [6]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2024-09-11 10:04:45--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 188.185.79.172, 188.184.98.238, 188.184.103.159, ...
Connecting to zenodo.org (zenodo.org)|188.185.79.172|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/1199620/files/SO_vectors_200.bin [following]
--2024-09-11 10:04:46--  https://zenodo.org/records/1199620/files/SO_vectors_200.bin
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1’

SO_vectors_200.bin? 100%[===================>]   1.35G  23.5MB/s    in 68s     

2024-09-11 10:05:54 (20.5 MB/s) - ‘SO_vectors_200.bin?download=1’ saved [1453905423/1453905423]



In [7]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [8]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [10]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место?

In [11]:
import numpy as np
# method most_simmilar
def cosine_sim(v1, v2):
    return np.array(v1 @ v2.T / np.linalg.norm(v1) / np.linalg.norm(v2))

In [19]:
sims = sorted(wv_embeddings.index_to_key, key=lambda w: cosine_sim(wv_embeddings[w], wv_embeddings[word]), reverse=True)[:6]
print(sims)

['dog', 'animal', 'dogs', 'mammal', 'cats', 'animals']


### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [20]:
import numpy as np
import re
from nltk.tokenize import WordPunctTokenizer
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)
tokenizer = WordPunctTokenizer()

In [84]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении

        return: векторное представление для вопроса
    """

    L = []
    for word in tokenizer.tokenize(question):
        if word in embeddings:
            L.append(embeddings[word])
    if len(L) == 0:
        return np.zeros(dim)
    return np.sum(L, axis=0)

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [85]:
question_to_vec('I love neural networks', wv_embeddings, tokenizer)

array([ -3.0426824 ,  -5.0673785 ,  -3.8562367 ,  -4.1130905 ,
         0.477499  ,   0.20120883,   3.356524  ,  -0.21155286,
         1.6159108 ,  -1.1141734 ,   0.8673905 ,   3.09375   ,
        -8.596398  ,  -3.957862  ,   6.6294885 ,   3.5036016 ,
         3.2525854 ,   5.540168  ,  -2.9620142 ,   2.6162837 ,
        -0.51658875,   2.2151704 ,  -2.348348  ,  -2.6483607 ,
        -1.8457832 ,  -0.47617072,  -1.8512394 ,   1.100045  ,
         6.04126   ,   0.7882416 ,  -4.485037  ,  -4.1601453 ,
        -3.6149812 ,  -3.049851  ,   8.286352  ,   5.176714  ,
        -0.57114536,  -1.7698342 ,  -0.3423465 ,  -5.6671057 ,
        -5.3650513 ,  -0.61261153,  -2.1186101 ,   7.467102  ,
        -0.4635815 ,   3.376504  ,   0.7832506 ,   3.9459739 ,
        -1.7300258 ,  -0.99814236,   5.0294704 ,   2.7368999 ,
        -2.096404  ,   7.125766  ,  -5.290267  ,   9.413416  ,
         3.6729612 ,  -3.1663244 ,   2.3692713 ,   0.13212073,
         3.4869137 ,  -1.3434069 ,  -2.7449448 ,   0.04

In [86]:
np.round(question_to_vec('I love neural networks', wv_embeddings, tokenizer)[2], 2)

-3.86

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями.

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера.
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [87]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    hits_value = np.sum([1 if rank <= k else 0 for rank in dup_ranks])
    return hits_value

In [88]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    dcg_value = np.sum([1 / np.log2(1 + rank) if rank <= k else 0 for rank in dup_ranks])
    return dcg_value

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [89]:
import pandas as pd

In [90]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0, 1, 1, 1]
Ваш ответ DCG: [0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [91]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [32]:
!unzip stackoverflow_similar_questions.zip

Archive:  stackoverflow_similar_questions.zip
   creating: data/
  inflating: data/.DS_Store          
   creating: __MACOSX/
   creating: __MACOSX/data/
  inflating: __MACOSX/data/._.DS_Store  
  inflating: data/train.tsv          
  inflating: data/validation.tsv     


Считайте данные.

In [92]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line)
    return data

Нам понадобиться только файл validation.

In [93]:
validation_data = read_corpus('./data/validation.tsv')

Кол-во строк

In [94]:
len(validation_data)

3760

Размер нескольких первых строк

In [95]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 54517
2 52818
3 54464
4 53724
5 52297


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [96]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [97]:
def index2d(arr, value):
    for i in range(len(arr)):
        if len(arr[i]) == len(value):
            flag = True
            for j in range(len(arr[i])):
                if arr[i][j] != value[j]:
                    flag = False
                    break
            if flag:
              return i
    raise ValueError("В массиве нет такого значения")

In [98]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    qvec = question_to_vec(question, embeddings, tokenizer)
    cvec = [question_to_vec(candidate, embeddings, tokenizer) for candidate in candidates]
    sorted_list = sorted(cvec, key=lambda vec: cosine_sim(vec, qvec), reverse=True)
    return [(index2d(cvec, candidate), candidates[index2d(cvec, candidate)]) for candidate in sorted_list]

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [99]:
questions = ['converting string to list', 'Sending array via Ajax fails']

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],

              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [100]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks)
        print()

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(1, 'WPF- How to update the changes in list item of a list'), (0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [101]:
# должно вывести
'''
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(*, 'Getting all list items of an unordered list in PHP'), #скрыт
            (*, 'select2 not displaying search results'), #скрыт
            (*, 'WPF- How to update the changes in list item of a list')]] #скрыт
'''

"\nresults = [[(1, 'C# create cookie from string and send it'),\n            (0, 'Convert Google results object (pure js) to Python object'),\n            (2, 'How to use jQuery AJAX for an outside domain?')],\n           [(*, 'Getting all list items of an unordered list in PHP'), #скрыт\n            (*, 'select2 not displaying search results'), #скрыт\n            (*, 'WPF- How to update the changes in list item of a list')]] #скрыт\n"

In [102]:
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(1, 'WPF- How to update the changes in list item of a list'), #скрыт
            (0, 'Getting all list items of an unordered list in PHP'), #скрыт
            (2, 'select2 not displaying search results')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [103]:
from tqdm.notebook import tqdm

In [131]:
wv_ranking = []
max_validation_examples = 100
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

<ipython-input-11-f4f361a89c93>:4: RuntimeWarning: invalid value encountered in scalar divide
  return np.array(v1 @ v2.T / np.linalg.norm(v1) / np.linalg.norm(v2))


In [132]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 94.000 | Hits@   1: 94.000
DCG@   5: 96.149 | Hits@   5: 98.000
DCG@  10: 96.149 | Hits@  10: 98.000
DCG@ 100: 96.604 | Hits@ 100: 100.000
DCG@ 500: 96.604 | Hits@ 500: 100.000
DCG@1000: 96.604 | Hits@1000: 100.000


### Эмбеддинги, обученные на корпусе похожих вопросов

In [115]:
train_data = read_corpus('./data/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [126]:
words = []
for data in train_data:
    p = data.split('\t')
    words.append((p[0], p[1][:-1]))
print(words[:5])

[('converting string to list', 'Convert Google results object (pure js) to Python object'), ('Which HTML 5 Canvas Javascript to use for making an interactive drawing tool?', 'Event handling for geometries in Three.js?'), ('Sending array via Ajax fails', 'Getting all list items of an unordered list in PHP'), ('How to insert CookieCollection to CookieContainer?', 'C# create cookie from string and send it'), ('Updating one element of a bound Observable collection', 'WPF- How to update the changes in list item of a list')]


In [130]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words, # data for model to train on
                 vector_size=200,                 # embedding vector size
                 min_count=5,             # consider words that occured at least 5 times
                 window=5).wv

In [133]:
wv_ranking = []
max_validation_examples = 100
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

<ipython-input-11-f4f361a89c93>:4: RuntimeWarning: invalid value encountered in scalar divide
  return np.array(v1 @ v2.T / np.linalg.norm(v1) / np.linalg.norm(v2))


In [134]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 100.000 | Hits@   1: 100.000
DCG@   5: 100.000 | Hits@   5: 100.000
DCG@  10: 100.000 | Hits@  10: 100.000
DCG@ 100: 100.000 | Hits@ 100: 100.000
DCG@ 500: 100.000 | Hits@ 500: 100.000
DCG@1000: 100.000 | Hits@1000: 100.000


### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:
